<a target="_blank" href="https://colab.research.google.com/github/victorlymarev/pandas/blob/main/notebooks/14-dates.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Работа с датами
#### [Ссылка на видео](https://youtu.be/XALU2vi2J7Q)

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
path_ltc_sample = '../tables/ltc_sample.parquet' if os.path.exists('../tables/ltc_sample.parquet') else 'https://drive.google.com/uc?id=1XaThogOOqKjJj50LvfJ9WqutjMAC5AxA'

ltc = pd.read_parquet(path_ltc_sample)
ltc.head()

In [ ]:
ltc['date'].head()

Приведение дат к целым числам

In [ ]:
ltc['date'].head().astype('int64')

In [ ]:
ltc['date'].head().astype(str)

Приведение строк и чисел к датам

In [ ]:
ltc['date'].head().astype(str).astype('datetime64')

In [ ]:
pd.to_datetime(ltc['date'].astype(str).head())

In [ ]:
pd.to_datetime(ltc['date'].astype('int64').head(), unit='ns')

### Разложение даты и времени на части

Но у дат есть свои специфические методы и атрибуты, которые можно получить через атрибут dt

| Название атрибута                          | Описание                                                |
|:------------------------------------------:|:-------------------------------------------------------:|
| year                                       | Год                                                     |
| month                                      | Месяц                                                   |
| day                                        | День                                                    |
| hour                                       | Час                                                     |
| minute                                     | Минута                                                  |
| second                                     | Секунда                                                 |
| microsecond                                | Микросекунда                                            |
| nanosecond                                 | Наносекунда                                             |
| date                                       | Дата (без учета времени и таймзоны)                     |
| time                                       | Время (без учета таймзоны)                              |
| timetz                                     | Время (с учетем таймзоны)                               |
| dayofyear или day_of_year                  | Номер дня в году (начиная с 1)                          |
| weekofyear или week или isocalendar().week | Номер недели в году (начиная с 1)                       |
| day_of_week или weekday                    | Номер дня недели (начиная с 0)                          |
| day_name('rus')                            | Название дня недели (на русском языке)                  |
| quarter                                    | Номер квартала (начиная с 1)                            |
| days_in_month или daysinmonth              | Число дней в месяце                                     |
| freq                                       | Частота (если есть)                                     |
| is_month_start                             | Проверка на то, что число - это первый день месяца      |
| is_month_end                               | Проверка на то, что число - это последний день месяца   |
| is_quarter_start                           | Проверка на то, что число - это первый день квартала    |
| is_quarter_end                             | Проверка на то, что число - это последний день квартала |
| is_year_start                              | Проверка на то, что число - это первый день года        |
| is_year_end                                | Проверка на то, что число - это последний день года     |
| is_leap_year                               | Проверка на то, что год - высокосный                    |
| tz                                         | Таймзона (если есть)                                    |
| isocalendar()                              | Разложение даты на год, неделю и день                   |

#### Год

In [ ]:
ltc['date'].dt.year

#### Месяц

In [ ]:
ltc['date'].dt.month

#### Номер дня недели

In [ ]:
ltc['date'].dt.weekday

#### Дата

Тип данных - object!

In [ ]:
ltc['date'].dt.date

#### Название дня недели 

In [ ]:
ltc['date'].dt.day_name()

In [ ]:
ltc['date'].dt.day_name('rus')

#### Проверка на то, что дата - начало месяца

In [ ]:
ltc['date'].dt.is_month_start

#### Установка часового пояса

In [ ]:
tz = ltc['date'].dt.tz_localize('Europe/Moscow')
tz

In [ ]:
url_timezones = 'https://en.wikipedia.org/wiki/List_of_tz_database_time_zones'
pd.read_html(url_timezones)[0].droplevel(0, 1)['TZ identifier'].tolist()#[:5]

#### Изменение часового пояса

In [ ]:
tz.head()

In [ ]:
tz.dt.tz_convert('Asia/Yekaterinburg').head()

### Округление дат

#### dt.normalize()

Приведение даты к полуночи

In [ ]:
ltc['date'].dt.normalize()

### dt.round()

Округление даты

In [ ]:
ltc['date'].dt.round(freq='h')

In [ ]:
ltc['date'].dt.round(freq='2d')

In [ ]:
# Это работать не будет
# ltc['date'].dt.round(freq='m')

Можно написать свое условие

In [ ]:
np.where(ltc['date'].dt.day >= 16,
         ltc['date'].dt.normalize() + pd.offsets.MonthEnd(0), # приводим дату к концу месяца если условие выполнено
#          Приводим дату к концу предыдущего месяца если условие не соблюдается
         ltc['date'].dt.normalize() + pd.offsets.MonthEnd(0) - pd.offsets.MonthEnd(1))

### dt.floor

Округление даты вниз

In [ ]:
ltc['date'].dt.floor(freq='d')

### dt.ceil

Округление даты вверх

In [ ]:
ltc['date'].dt.ceil(freq='d')

### dt.strftime()
Представление даты в виде строки нужного формата

In [ ]:
ltc['date'].head().dt.strftime('%d-%m-%Y')

### С датами можно использовать агрегационные функции

In [ ]:
ltc['date'].mean()

In [ ]:
ltc['date'].std()

In [ ]:
ltc['date'].astype('int64').mean().astype('int64').astype('datetime64[ns]')

### Если мы работаем не с Series, то ставить dt не надо

In [ ]:
pd.Timestamp('2023-07-23')

In [ ]:
pd.Timestamp('2023-07-23').day

In [ ]:
pd.Timestamp('2023-07-23').month

In [ ]:
pd.date_range('2023-07-23', '2023-08-23', freq='d').day_name('rus')

#### Все эти методы можно использовать при фильтрации строк

In [ ]:
ltc.query('date.dt.is_month_start').head()

### asof

#### Метод возвращает актуальное (последнее доступное) на указанную дату заначение

In [ ]:
(ltc
    .set_index('date')
    .asof('2017-03-30 23:05:00')
)

In [ ]:
ltc.query('date <= "2017-03-30 23:05:00"').sort_values(by='date').tail(1)

### at_time

#### Метод возвращает все значения, произошедшие в определенное время, без учета даты

In [ ]:
(ltc
    .set_index('date')
    .at_time('23:00')
    .head()
)

### between_time

#### Метод возвращает все значения между двумя значениями времени без учета даты

In [ ]:
(ltc
    .set_index('date')
    .between_time('23:00', '04:00')
    .head()
)

### Last
#### Метод возвращает все послединие значения за какой-то интервал (например, за час)

In [ ]:
# все значения за поледний час
(ltc
    .set_index('date')
    .last('2d')
)

### First
#### Метод возвращает все первые значения за какой-то интервал (например, за час)

In [ ]:
# все значения за первый год (Календарный)
(ltc
    .set_index('date')
    .first('Y')
)

### Список сокращений (регистр не важен)

| Сокращение   | Расшифровка                              | Сокращение   | Расшифровка                    |
|:-------------|:-----------------------------------------|:-------------|:-------------------------------|
| B            | Последний рабочий день в неделе          | QS           | Начало квартала                |
| C            | Пользовательский последний день в неделе | BQS          | Первый рабочий день квартала   |
| D            | День                                     | A, Y         | Конец года                     |
| W            | Неделя                                   | BA, BY       | Последний рабочий день в году  |
| M            | Конец месяца                             | AS, YS       | Начало года                    |
| SM           | 15 число месяца                          | BAS, BYS     | Первый рабочий день в году     |
| BM           | Последний рабочий день в месяце          | BH           | Рабочий час (от 9:00 до 17:00) |
| CBM          | Пользовательский конец месяца            | H            | Час                            |
| MS           | Первое число месяца                      | T, min       | Минута                         |
| SMS          | 1 и 15 число месяца                      | S            | Секунда                        |
| BMS          | Первый рабочий день месяца               | L, ms        | Миллисекунда                   |
| CBMS         | Пользовательский первый день месяца      | U, us        | Микросекунда                   |
| Q            | Конец квартала                           | N            | Наносекунда                    |
| BQ           | Последний рабочий день квартала          | nan          | nan                            |

## Арифметические опериции

### Timedelta

Используется в арифметических операциях с датами

In [ ]:
pd.Timedelta(1, 'day')

In [ ]:
pd.Timedelta(135, 'min')

In [ ]:
pd.Timedelta(9, 's')

In [ ]:
pd.Timedelta(0.25, 'h')

### Возможные варианты сокращений

* **'W'** - неделя, **'D'** - день, **'T'** - минута, **'S'** - секунда, **'L'** - милисекунда, **'U'** - микросекунда, **'N'** - наносекунда
* **'days'** или **'day'** - день
* **'hours'**, **'hour'**, **'hr'**, или **'h'** - час
* **'minutes'**, **'minute'**, **'min'**, или **'m'** - минута
* **'seconds'**, **'second'**, или **'sec'** - секунда
* **'milliseconds'**, **'millisecond'**, **'millis'**, или **'milli'** - милисекунда
* **'microseconds'**, **'microsecond'**, **'micros'**, или **'micro'** - микросекунда
* **'nanoseconds'**, **'nanosecond'**, **'nanos'**, **'nano'**, или **'ns'** - наносекунда

In [ ]:
# можно исользовать:
# days, hours, minutes, seconds, milliseconds, microseconds, nanoseconds
pd.Timedelta(days=3, hours=4, minutes=122, seconds=912)

In [ ]:
pd.Timedelta(135, 'min') * 10

In [ ]:
pd.Timedelta(135, 'min') / 10

In [ ]:
pd.Timedelta(135, 'min') + pd.Timedelta(9, 's')

In [ ]:
pd.Timedelta(135, 'min') - pd.Timedelta(9, 's')

### to_timedelta

Преобразует набор чисел в объекты timedelta

In [ ]:
pd.to_timedelta(np.arange(10), 'days')

In [ ]:
pd.to_timedelta([1, 12.8, 11.5, 0.3], 'days')

#### Пложим несколько дат в отдельную переменную

In [ ]:
ts = ltc.loc[[300, 341, 342], 'date'].copy()
ts

### Опрации с Timedelta

In [ ]:
ts + pd.Timedelta(8, 'h')

In [ ]:
ts - pd.Timedelta(8, 'h')

In [ ]:
ts - 10 * pd.Timedelta(8, 'h')

### Offsets

In [ ]:
dir(pd.offsets)[:42]

### MonthEnd

#### Приведем дату к концу месяца

In [ ]:
ts

In [ ]:
ts + pd.offsets.MonthEnd(0)

In [ ]:
ts.dt.normalize() + pd.offsets.MonthEnd(0)

Если передать 1, то все даты, что не конец месяца - станут концом месяца, но если дата - конец месяца, то она станет концом следующего месяца

In [ ]:
ts.dt.normalize() + pd.offsets.MonthEnd(1)

Приведем даты к концу следующего месяца

In [ ]:
ts

In [ ]:
ts.dt.normalize() + pd.offsets.MonthEnd(0) + pd.offsets.MonthEnd(1)

Если вычесть pd.offsets.MonthEnd(0) то это будет эквивалентно прибавлению

In [ ]:
ts.dt.normalize()

In [ ]:
ts.dt.normalize() - pd.offsets.MonthEnd(0)

Такая запись приведет даты к концу предыдущего месяца

In [ ]:
ts.dt.normalize() - pd.offsets.MonthEnd(1)
# ts.dt.normalize() + pd.offsets.MonthEnd(0) - pd.offsets.MonthEnd(1)

Приведение даты к концу предпредыдущего месяца

In [ ]:
ts.dt.normalize() - pd.offsets.MonthEnd(2)
# ts.dt.normalize() + pd.offsets.MonthEnd(0) - pd.offsets.MonthEnd(2)

### Приведение даты к началу месяца

In [ ]:
ts.dt.normalize() + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)

### Добавляем 9 рабочих дней

In [ ]:
ts

In [ ]:
ts + pd.offsets.BusinessDay(9)

#### Добавляем 9 рабочих дней при четырехдневной рабочей неделе

In [ ]:
ts + pd.offsets.CustomBusinessDay(9, weekmask='Mon Tue Wed Thu')

Добавим 10 рабочих часов. Рабочие часы Пн-Пт с 9 до 17

In [ ]:
ts

In [ ]:
ts + pd.offsets.BusinessHour(10)

### Метод astype

Метод приведит дату к началу периода

In [ ]:
ts

In [ ]:
ts.dt.date.astype('datetime64[W]')

In [ ]:
ts.dt.date.astype('datetime64[M]')

In [ ]:
ts.dt.date.astype('datetime64[Y]')

# Задания

#### Описание таблиц лежит [здесь](https://github.com/victorlymarev/pandas/tree/main/tables#%D0%BE%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D0%B5-%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86)

Некоторые таблицы занимают много памяти, поэтому каждые 5-10 заданий лучше перезапускайте ноутбук.

В формулировке некоторых заданий может содержаться вариативность. Если у вас есть сомнения, что требуется в задании, попробуйте решить несколько вариантов. Если вы не понимаете задание, можете написать об этом в комментариях под видео.

В большинстве случаев внешний вид итоговой таблицы не определен. Выведите на экран наиболее адекватный вариант. То есть таблицу, из которой можно будет сделать выводы.

Курс пока находится в разработке. Вы можете помочь другим людям добавив свое решение [сюда](https://docs.google.com/forms/d/1HYTBz_KfssY3Jps2dC3n0YnEqa6WBb5OIhLo1d32Xzw/edit).

Посмотреть решения других людей можно [здесь](https://docs.google.com/spreadsheets/d/1pMDYO-9UneLbPPnEuQ2shig0TOJdQTU-zipifyAnZMk/edit?resourcekey#gid=1998648012)

### Задание 1

Магазин открывается через 15 дней после завершения ремонта. Найдите дату, когда был закончен ремонт в каждом магазине

In [ ]:
import os
import pandas as pd

path_shops = '../tables/shops.xlsx' if os.path.exists('../tables/shops.xlsx') else 'https://drive.google.com/uc?id=1gfnmceJa3Mc1X06NftTx9G9QfKfprjEB'

shops = pd.read_excel(path_shops)
shops.head()

In [ ]:
# напишите свой код здесь

### Задание 2

На каждую отчетную дату найдите возраст сотрудника в годах

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задание 3

Округлите дату покупки до недели (при помощи метода astype)

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задание 4

Для каждой строки найдите день недели на русском языке, когда была совершена покупка

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задание 5

Найдте цену на litecoin по состоянию на 29 мая 2020 года 18:32

In [ ]:
# таблица ltc - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_ltc_sample на path_ltc_full внутри функции read_parquet

import os
import pandas as pd

# path_ltc_full = '../tables/ltc.parquet' if os.path.exists('../tables/ltc.parquet') else "https://drive.usercontent.google.com/download?id=1ZkAmVZverOV3aGwmEQGAFXgXnQ6pPsZw&export=download&authuser=1&confirm=t&uuid=b827b3e2-7c5d-4979-9d25-f1c34954ac9f&at=APZUnTUs_oUnCQujGIlgn2Zkb5VG:1693327327264"
path_ltc_sample = '../tables/ltc_sample.parquet' if os.path.exists('../tables/ltc_sample.parquet') else 'https://drive.google.com/uc?id=1XaThogOOqKjJj50LvfJ9WqutjMAC5AxA'

ltc = pd.read_parquet(path_ltc_sample)
ltc.head()

In [ ]:
# напишите свой код здесь

### Задание 6

Приведите колонку purchase_date к началу года

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задание 7

Для каждого сотрудника посчитайте посчитайте сколько было ему лет, когда он впервые устроился на работу.

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь

### Задание 8

Найдите цену закрытия за каждый день (цена закрытия - последняя цена за определенный период)

In [ ]:
# таблица ltc - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_ltc_sample на path_ltc_full внутри функции read_parquet

import os
import pandas as pd

# path_ltc_full = '../tables/ltc.parquet' if os.path.exists('../tables/ltc.parquet') else "https://drive.usercontent.google.com/download?id=1ZkAmVZverOV3aGwmEQGAFXgXnQ6pPsZw&export=download&authuser=1&confirm=t&uuid=b827b3e2-7c5d-4979-9d25-f1c34954ac9f&at=APZUnTUs_oUnCQujGIlgn2Zkb5VG:1693327327264"
path_ltc_sample = '../tables/ltc_sample.parquet' if os.path.exists('../tables/ltc_sample.parquet') else 'https://drive.google.com/uc?id=1XaThogOOqKjJj50LvfJ9WqutjMAC5AxA'

ltc = pd.read_parquet(path_ltc_sample)
ltc.head()

In [ ]:
# напишите свой код здесь

### Задание 9

Посчитайте среднее время покупки в магазине 9 21 окрября 2022 года. Результат округлити до минут. Кроме того, постройте гистограмму числа покупок по времени.

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_sample_check на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_sample_check = '../tables/sales_sample_check.parquet' if os.path.exists('../tables/sales_sample_check.parquet') else 'https://drive.google.com/uc?id=1oYT518oqGnEF51PSFHfSHYNP-690ktFL'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_sample_check)
sales.head()

In [ ]:
# напишите свой код здесь

### Задание 10

Посмотрите на рапределение людей с высшим образованием в разрезе возраста.

Измените образование с высшего на среднее общее если сотруднику на отчетную дату было меньше 21 года и месяц отчетной даты - январь, февраль, март, апрель, май или июнь

In [ ]:
import os
import pandas as pd

path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

In [ ]:
# напишите свой код здесь